In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [2]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 3
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [3]:
#might need to rewrite these

def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'airport':0,'shopping_mall':1,'metro_station':2,'street_pedestrian':3,'public_square':4,'street_traffic':5,'tram':6,'bus':7,'metro':8,'park':9}
labelRef={'inside':0, 'outside':1, 'transport':2}
def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[0].replace('logMelSpec\\','')
        if location in ['airport','shopping_mall','metro_station']:
            location='inside'
        elif location in ['street_pedestrian','public_square','street_traffic','park']:
            location='outside'
        else:
            location='transport'
        Y_[i,labelRef[location]]=1
    return X_, Y_
#trainlist,validationlist,testlist=fileLists()
#X_train, Y_train = getData(trainlist)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


 data sorted


In [5]:
###### Convolutional model
def compileCRNN(cols,rows,nb_classes=1):
    model = Sequential()
    # conv1 layer
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=(cols,rows,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(64, (7, 7), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,7), strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    model.add(Flatten())
    # fc1 layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(prob_drop_hidden))
    model.add(BatchNormalization())
   
    # fc2 layer
    model.add(Dense(nb_classes, activation='softmax'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = compileCRNN(cols,rows,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 216, 64)       256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 108, 64)       0         
__________

In [6]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\3groupedScenes', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [7]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 3
rows, cols = 431, 128  
nb_epoch = 200


trainlist,validationlist,testlist=fileLists()
print('lists sorted')
X_tr, Y_train = getData(trainlist)
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_train=np.expand_dims(X_tr,axis=3)
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)
print(X_train.shape, X_val.shape, X_test.shape)
print('data sorted')
model = compileCRNN(cols,rows,nb_classes=nb_classes)
#model = load_model('models\\sceneModel200.hdf5')
print('model compiled')
savemodelfilename='models\\3groupedScenes.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)
print(datetime.now())
print('model saved')

trainfiles:  6122
validationfiles:  2518
testfiles:  2518
lists sorted
data obtained
(6122, 128, 431, 1) (2518, 128, 431, 1) (2518, 128, 431, 1)
data sorted
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_9 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 216, 64)   

6122/6122 [==============================] - 23s 4ms/step - loss: 0.2370 - acc: 0.9162 - val_loss: 2.7808 - val_acc: 0.6120
Epoch 41/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2225 - acc: 0.9165 - val_loss: 2.0684 - val_acc: 0.6048
Epoch 42/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2628 - acc: 0.9025 - val_loss: 0.6689 - val_acc: 0.8129
Epoch 43/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2251 - acc: 0.9182 - val_loss: 2.4042 - val_acc: 0.6926
Epoch 44/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1988 - acc: 0.9306 - val_loss: 0.9008 - val_acc: 0.8237
Epoch 45/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.1827 - acc: 0.9355 - val_loss: 0.5188 - val_acc: 0.8403
Epoch 46/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1727 - acc: 0.9414 - val_loss: 0.8069 - val_acc: 0.8471
Epoch 47/200
6122/6122 [==============================

Epoch 100/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.0690 - acc: 0.9794 - val_loss: 1.0413 - val_acc: 0.8388
Epoch 101/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.0718 - acc: 0.9775 - val_loss: 2.3871 - val_acc: 0.6914
Epoch 102/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0655 - acc: 0.9771 - val_loss: 0.7451 - val_acc: 0.7836
Epoch 103/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.0798 - acc: 0.9714 - val_loss: 1.0446 - val_acc: 0.6513
Epoch 104/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0864 - acc: 0.9696 - val_loss: 4.1576 - val_acc: 0.6442
Epoch 105/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0644 - acc: 0.9786 - val_loss: 0.8506 - val_acc: 0.8102
Epoch 106/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0786 - acc: 0.9752 - val_loss: 0.4865 - val_acc: 0.8789
Epoch 107/200
6122/6122 [=========

6122/6122 [==============================] - 23s 4ms/step - loss: 0.0743 - acc: 0.9758 - val_loss: 3.2442 - val_acc: 0.7029
Epoch 160/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0629 - acc: 0.9820 - val_loss: 0.8904 - val_acc: 0.8423
Epoch 161/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0700 - acc: 0.9812 - val_loss: 0.5512 - val_acc: 0.8539
Epoch 162/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0926 - acc: 0.9691 - val_loss: 0.6120 - val_acc: 0.8495
Epoch 163/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0643 - acc: 0.9794 - val_loss: 2.7722 - val_acc: 0.7204
Epoch 164/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0761 - acc: 0.9742 - val_loss: 0.6812 - val_acc: 0.8384
Epoch 165/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0657 - acc: 0.9786 - val_loss: 1.4524 - val_acc: 0.8002
Epoch 166/200
6122/6122 [=======================